In [1]:
%pip install ultralytics tqdm numpy opencv-python

In [1]:
import os
import json
import cv2
from ultralytics import YOLO
import shutil
import numpy as np
import logging
from tqdm import tqdm
import re
import random
import string

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [3]:
current_working_directory = os.getcwd()

YOLO_path = os.path.join(current_working_directory, "models/yolov8n-pose.pt") #path to the YOLO model
datasetDir = os.path.join(current_working_directory, "archive", "RWF-2000") #change here to the other dataset like RFC-2000
newKeypointsDir = os.path.join(current_working_directory, "archive", "Keypoints_Dataset") #The new json data are saved here
os.makedirs(newKeypointsDir, exist_ok=True)

'c:\\Users\\gorme\\projects\\godseye\\apps\\backend\\dataset_processing'

In [ ]:
# Options for saving
save_original = True  # Set to True if you want to save the original video
save_processed = True  # Set to True if you want to save the processed video with keypoints

In [3]:

model = YOLO(YOLO_path) #staring the model

In [4]:
# Define dataset categories and video processing limits
sets = ['train', 'val']
categories = ['Fight', 'NonFight']
video_limit = 10  # Limit number of videos processed per category

#making the labels in the COCO format for YOLO model
labeledKeypoints = [
    "nose", "left_eye", "right_eye", "left_ear", "right_ear",
    "left_shoulder", "right_shoulder", "left_elbow", "right_elbow",
    "left_wrist", "right_wrist", "left_hip", "right_hip",
    "left_knee", "right_knee", "left_ankle", "right_ankle"
]

In [ ]:
# Helper function to sanitize filenames
def sanitize_filename(filename):
    # Remove any character that is not a letter, number, or underscore
    sanitized_name = re.sub(r'[^A-Za-z0-9_]', '', filename)
    
    # If the sanitized name is too short or empty, create a random name
    if len(sanitized_name) < 5:
        sanitized_name = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
    
    return sanitized_name

In [5]:
# Process each set (train/val) and category (Fight/NonFight)
for set_name in tqdm(sets, desc="Processing Sets"):
    for category in tqdm(categories, desc=f"Processing Categories in {set_name}", leave=False):
        input_dir = os.path.join(datasetDir, set_name, category)
        video_count = 0

        for video_file in tqdm(os.listdir(input_dir), desc=f"Processing Videos in {category}", leave=False):
            if not video_file.endswith(('.avi', '.mp4')):
                continue
            
            ## uncomment the code below to apply the video limit
            # if video_count >= video_limit:
            #     break

            # Sanitize or generate a unique name for problematic video filenames
            video_name = sanitize_filename(os.path.splitext(video_file)[0])
            video_dir = os.path.join(newKeypointsDir, set_name, category, f"{video_name}")
            os.makedirs(video_dir, exist_ok=True)

            video_path = os.path.join(input_dir, video_file)
            json_output_path = os.path.join(video_dir, f"{video_name}.json")

            # Copy original video if save_original is True
            if save_original:
                original_save_path = os.path.join(video_dir, f"{video_name}.avi")
                try:
                    shutil.copy2(video_path, original_save_path)
                    logging.info(f"Original video saved: {original_save_path}")
                except FileNotFoundError as e:
                    logging.warning(f"File not found or accessible: {video_path}. Skipping this file.")
                    continue

            video_capture = cv2.VideoCapture(video_path)
            frame_index = 0
            keyPointsData = []

            # Set up writer for processed video if save_processed is True
            if save_processed:
                processed_video_path = os.path.join(video_dir, f"{video_name}_processed.avi")
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                fps = video_capture.get(cv2.CAP_PROP_FPS)
                frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
                frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
                processed_video_writer = cv2.VideoWriter(processed_video_path, fourcc, fps, (frame_width, frame_height))
                logging.info(f"Processed video writer initialized for: {processed_video_path}")

            # Process each frame
            while video_capture.isOpened():
                ret, frame = video_capture.read()
                if not ret:
                    break

                # Detect keypoints using YOLO model
                results = model(frame)
                newFrameData = []

                # Extract keypoints and bounding box data for JSON output
                for result in results:
                    boxes = result.boxes
                    keypoints = result.keypoints

                    if boxes is not None and len(boxes) > 0:
                        for i in range(len(boxes)):
                            box_data = boxes.xyxy[i].cpu().numpy()
                            confidence = boxes.conf[i].cpu().item()
                            box = {
                                "x1": float(box_data[0]),
                                "y1": float(box_data[1]),
                                "x2": float(box_data[2]),
                                "y2": float(box_data[3])
                            }

                            keypoints_data = []
                            if keypoints is not None:
                                keypoints_array = keypoints.data[i].cpu().numpy()
                                for j, (x, y, conf) in enumerate(keypoints_array):
                                    keypoints_data.append({
                                        "label": labeledKeypoints[j],
                                        "coordinates": {"x": float(x), "y": float(y)},
                                        "confidence": float(conf)
                                    })

                            newFrameData.append({
                                "person_id": i + 1,
                                "confidence": confidence,
                                "box": box,
                                "keypoints": keypoints_data
                            })

                keyPointsData.append({"frame": frame_index, "detections": newFrameData})

                # Use results[0].plot() for processed video frame
                if save_processed:
                    frame_with_keypoints = results[0].plot()
                    processed_video_writer.write(frame_with_keypoints)

                frame_index += 1

            # Release resources
            video_capture.release()
            if save_processed:
                processed_video_writer.release()
                logging.info(f"Processed video saved to: {processed_video_path}")
                
            # Save JSON data for keypoints
            with open(json_output_path, 'w') as json_file:
                json.dump(keyPointsData, json_file, indent=4)
                logging.info(f"Keypoints data saved to: {json_output_path}")

            video_count += 1

print("Done")



0: 384x640 9 persons, 202.1ms
Speed: 12.8ms preprocess, 202.1ms inference, 15.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 126.8ms
Speed: 2.0ms preprocess, 126.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 167.5ms
Speed: 4.1ms preprocess, 167.5ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 137.7ms
Speed: 2.2ms preprocess, 137.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 156.6ms
Speed: 2.3ms preprocess, 156.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 146.0ms
Speed: 2.4ms preprocess, 146.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 200.5ms
Speed: 5.4ms preprocess, 200.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 208.6ms
Speed: 3.7ms preprocess, 208.6ms inference, 2.7ms postproce